In [2]:
!python3 -m pip install mwparserfromhell==0.5.4

    100% |████████████████████████████████| 143kB 8.8MB/s ta 0:00:01
  Running setup.py bdist_wheel for mwparserfromhell ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/2a/76/d5/7088b941df3b362c45dd7912dd05314bc034751ec9cbca9a75
Successfully built mwparserfromhell
You are using pip version 10.0.1, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import requests
from bs4 import BeautifulSoup
import os
import time
from keras.utils import get_file
try:
    from urllib.request import urlretrieve
except ImportError:
    from urllib import urlretrieve
import xml.sax

import subprocess
import re
import mwparserfromhell
import json

In [5]:
# check the latest wikipedia dumps (available for download - whole wikipedia as xml files!!!)
index = requests.get('https://dumps.wikimedia.org/enwiki/').text
index

'<html>\r\n<head><title>Index of /enwiki/</title></head>\r\n<body bgcolor="white">\r\n<h1>Index of /enwiki/</h1><hr><pre><a href="../">../</a>\r\n<a href="20190501/">20190501/</a>                                          21-Jun-2019 01:33                   -\r\n<a href="20190520/">20190520/</a>                                          02-Jul-2019 01:28                   -\r\n<a href="20190601/">20190601/</a>                                          21-Jul-2019 01:34                   -\r\n<a href="20190620/">20190620/</a>                                          02-Aug-2019 01:28                   -\r\n<a href="20190701/">20190701/</a>                                          08-Jul-2019 21:41                   -\r\n<a href="20190720/">20190720/</a>                                          25-Jul-2019 19:24                   -\r\n<a href="20190801/">20190801/</a>                                          07-Aug-2019 14:05                   -\r\n<a href="latest/">latest/</a>             

In [6]:
soup_index = BeautifulSoup(index, 'html.parser')

In [7]:
dumps = [a['href'] for a in soup_index.find_all('a') 
             if a.has_attr('href') and a.text[:-1].isdigit()]
dumps

['20190501/',
 '20190520/',
 '20190601/',
 '20190620/',
 '20190701/',
 '20190720/',
 '20190801/']

In [8]:
# find the newest of all those dumps which is already finished 
for dump_url in sorted(dumps, reverse=True):
    print(dump_url)
    dump_html = index = requests.get('https://dumps.wikimedia.org/enwiki/' + dump_url).text
    soup_dump = BeautifulSoup(dump_html, 'html.parser')
    pages_xml = [a['href'] for a in soup_dump.find_all('a') 
                 if a.has_attr('href') and a['href'].endswith('-pages-articles.xml.bz2')]
    if pages_xml:
        break
    time.sleep(0.8)

20190801/


In [9]:
wikipedia_dump = pages_xml[0].rsplit('/')[-1]
url = url = 'https://dumps.wikimedia.org/' + pages_xml[0] 
wikipedia_dump, url

('enwiki-20190801-pages-articles.xml.bz2',
 'https://dumps.wikimedia.org//enwiki/20190801/enwiki-20190801-pages-articles.xml.bz2')

In [ ]:
path = get_file(wikipedia_dump, url)
path

In [7]:
def process_article(title, text):
    rotten = [(re.findall('\d\d?\d?%', p), re.findall('\d\.\d\/\d+|$', p), p.lower().find('rotten tomatoes')) for p in text.split('\n\n')]
    rating = next(((perc[0], rating[0]) for perc, rating, idx in rotten if len(perc) == 1 and idx > -1), (None, None))
    wikicode = mwparserfromhell.parse(text)
    film = next((template for template in wikicode.filter_templates() 
                 if template.name.strip().lower() == 'infobox film'), None)
    if film:
        properties = {param.name.strip_code().strip(): param.value.strip_code().strip() 
                      for param in film.params
                      if param.value.strip_code().strip()
                     }
        links = [x.title.strip_code().strip() for x in wikicode.filter_wikilinks()]
        return (title, properties, links) + rating

In [8]:
class WikiXmlHandler(xml.sax.handler.ContentHandler):
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._movies = []
        self._curent_tag = None

    def characters(self, content):
        if self._curent_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        if name in ('title', 'text'):
            self._curent_tag = name
            self._buffer = []

    def endElement(self, name):
        if name == self._curent_tag:
            self._values[name] = ' '.join(self._buffer)

        if name == 'page':
            movie = process_article(**self._values)
            if movie:
                self._movies.append(movie)

In [16]:
parser = xml.sax.make_parser()
handler = WikiXmlHandler()
parser.setContentHandler(handler)
for line in subprocess.Popen(['bzcat'], stdin=open(path), stdout=subprocess.PIPE).stdout:
    try:
        parser.feed(line)
    except StopIteration:
        break

In [20]:
with open('generated/wp_movies.ndjson', 'wt') as fout:
    for movie in handler._movies:
         fout.write(json.dumps(movie) + '\n')